In [1]:
from mip import Model, xsum, maximize, BINARY, OptimizationStatus,minimize

import pandas as pd; import numpy as np; 

import time


In [2]:
import os 
#выбор рабочей папки и данных
folder="""C:\\КОНКУРС\\"""
os.chdir(folder)
#Входные данные
table_in="""train25.xlsx"""
sheet='data25'
#структура данных признаки+метки болезней (болезней несколько)
#задаем число признаков
p=25
#Выходные данные
table_out="""kom7_p25_Z3_Z5.xlsx"""
#максимальное время счета
time_max=1200

#---- ЗАГРУЗКА ДАННЫХ ----
df=pd.read_excel(table_in,sheet_name=sheet)
print(df.head())


   X1  X2  X3  X4  X5  X6  X7  X8  X9  X10  ...  X19  X20  X21  X22  X23  X24  \
0   1   1   1   3   2   0   0   0   0    0  ...    0    0    0    0    0    0   
1   0   0   1   5   4   0   0   1   1    1  ...    2    0    1    1    0    0   
2   0   1   1   5   4   0   0   0   1    0  ...    1    1    0    0    0    0   
3   1   1   0   3   2   0   0   0   1    0  ...    1    1    0    0    0    0   
4   0   1   1   3   3   0   1   1   1    1  ...    1    0    0    0    0    0   

   X25  Z3  Z4  Z5  
0    0   0   0   0  
1    0   0   0   0  
2    0   0   0   0  
3    0   0   0   0  
4    0   1   1   0  

[5 rows x 28 columns]


In [3]:
#формируем список заболеваний
Z_col=df.columns[p:].values
z=len(Z_col)

In [4]:
df[df.columns[:p]]

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25
0,1,1,1,3,2,0,0,0,0,0,...,2,18,8.00,0,0,0,0,0,0,0
1,0,0,1,5,4,0,0,1,1,1,...,0,0,4.00,2,0,1,1,0,0,0
2,0,1,1,5,4,0,0,0,1,0,...,2,17,8.00,1,1,0,0,0,0,0
3,1,1,0,3,2,0,0,0,1,0,...,2,13,8.00,1,1,0,0,0,0,0
4,0,1,1,3,3,0,1,1,1,1,...,2,16,7.00,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,1,1,1,4,3,0,0,0,0,1,...,2,18,8.00,0,0,0,0,0,0,0
951,0,0,1,3,2,0,0,0,1,0,...,1,26,9.75,0,1,0,0,1,0,0
952,0,1,1,3,3,1,0,0,1,0,...,2,20,9.50,1,1,1,0,1,0,0
953,0,1,0,5,3,0,0,0,1,0,...,2,21,9.50,1,2,0,0,1,1,0


In [5]:
#формируем матрицу признакров со свободным членом + Свободный член
MP=df[df.columns[0:p]].assign(p0 = 1)

# Свободный член
MP=MP.fillna(0)
print(MP.head())

   X1  X2  X3  X4  X5  X6  X7  X8  X9  X10  ...  X17  X18  X19  X20  X21  X22  \
0   1   1   1   3   2   0   0   0   0    0  ...   18  8.0    0    0    0    0   
1   0   0   1   5   4   0   0   1   1    1  ...    0  4.0    2    0    1    1   
2   0   1   1   5   4   0   0   0   1    0  ...   17  8.0    1    1    0    0   
3   1   1   0   3   2   0   0   0   1    0  ...   13  8.0    1    1    0    0   
4   0   1   1   3   3   0   1   1   1    1  ...   16  7.0    1    0    0    0   

   X23  X24  X25  p0  
0    0    0    0   1  
1    0    0    0   1  
2    0    0    0   1  
3    0    0    0   1  
4    0    0    0   1  

[5 rows x 26 columns]


### Записываем переменные и константы

In [ ]:
#Запись решения в эксель
#Открываем сессию записи в эксель
writer = pd.ExcelWriter(folder+table_out,engine='xlsxwriter')

for it_z in Z_col:
    
    it_sheet=sheet+'_'+it_z

    MP_cycle=MP.assign(Z = df[it_z])

    start=time.time()
    # Создаем модель
    model = Model('Linear') 

    #Записываем константы

    L=10000
    e=0.1
    t_num=7

    #Записываем размер матрицы
    I=range(len(MP_cycle.columns)-1) # колонки

    J=range(len(MP_cycle)) # строки

    T=range(t_num) #число членов комитета

    #Записываем переменные
    x=[[model.add_var(var_type='C', lb=-L, ub=L) for i in I] for t in T]

    z=[[model.add_var(var_type='B') for j in J] for t in T]

    d=[model.add_var(var_type='B') for j in J]

    g=model.add_var(var_type='B')

    q=model.add_var(var_type='B')


    #Записываем массив признаков для каждого класса
    P1=np.array(MP_cycle.loc[MP_cycle["Z"]==0].drop("Z",axis=1))
    k1=len(P1)
    J1=range(k1) # строки0

    P2=np.array(MP_cycle.loc[MP_cycle["Z"]==1].drop("Z",axis=1))
    k2=len(P2)
    #!!! Особое внимание как записывается массив индексов, так как класс 2 начинается после окончания строк класса 1, 
    #!!! то массив стартует с k1
    J2=range(k1,k1+k2) # строки

    # Упорядоченный массив всех признаков сначала Класс 1, потом Класс 2 (так короче и понятней будет запись в кодах питона)
    P=np.vstack((P1,P2))

    #Записываем ограничения
    for t in T:
        model += x[t][0]==g-q
        model += g+q==1

    for t in T:
        for j1 in J1:
            model +=xsum(x[t][i]*P[j1][i] for i in I)>=-L*z[t][j1]+e

    for t in T:
        for j2 in J2:
            model +=xsum(x[t][i]*P[j2][i] for i in I)<=L*z[t][j2]-e

    #!!! Важно не забыть правильно указать индексы строк 
    for j1 in J1:
        model +=xsum(z[t][j1] for t in T) <=t_num*d[j1]

    for j2 in J2:
        model +=xsum(z[t][j2] for t in T)<=t_num-1+t_num*d[j2]

    # Записываем целевую функцию
    model.objective = minimize( 
        k2*xsum(d[j1] for j1 in J1)+
        k1*xsum(d[j2] for j2 in J2)
    )


    # Оптимизируем
    status=model.optimize(max_seconds=time_max) # ограничили время оптимизации max_t секундами

    # Сохраняем результаты решения
    x_val=[]

    for t in T:
        x_val_sub=[]
        for i in I:
            x_val_sub.append(x[t][i].x)

        x_val.append(x_val_sub)


    print('Итоговое время на решение задачи LP = ',time.time()-start,' сек.')

    # Вывод значения оптимизируемой величины
    print('optimal solution cost {}'.format(model.objective_value))


    # Считаем число нарушений условий комитета
    # Вычисляем показатели качества решения
    d1_val=[]

    for j1 in J1:
        d1_val.append(d[j1].x)

    # Вывод суммы d1
    fn=np.array(d1_val).sum()

    print('FN=',fn)


    d2_val=[]

    for j2 in J2:
        d2_val.append(d[j2].x)

    # Вывод суммы d2
    fp=np.array(d2_val).sum()

    print('FP=',fp)

    tp=k1-fn
    tn=k2-fp

    accuracy=(tp+tn)/(k1+k2)
    precision=tp/(tp+fp)
    tpr=tp/k1
    fpr=fp/k2
    auc=(1+tpr-fpr)/2

    print('ACCURASY=',accuracy)
    print('PRECISION=',precision)
    print('TPR=',tpr)
    print('FPR=',fpr)
    print('AUC=',auc)
    print('------------------------------------')
    
    # Записываем коэффициенты
    pd.DataFrame(data=x_val,columns=['x'+str(x+1) for x in I]).to_excel(writer,index=None,sheet_name=it_sheet)
    # Записываем метрики
    pd.DataFrame(data=[k1,k2,fn,fp,accuracy,precision,tpr,fpr,auc],index=["K1","K2","FN","FP","Accuracy","Precision","TPR","FPR",'AUC'],columns=['Значение']).to_excel(writer,index=True,startrow=len(x_val)+3,sheet_name=it_sheet)

# Закрываем сессию вне цикла
writer.save()

Итоговое время на решение задачи LP =  1204.1364409923553  сек.
optimal solution cost 53274.0
FN= 176.0
FP= 39.0
ACCURASY= 0.774869109947644
PRECISION= 0.9443651925820257
TPR= 0.7899761336515513
FPR= 0.3333333333333333
AUC= 0.7283214001591091
------------------------------------
